In [1]:
import plotly.graph_objects as go
import re

import pandas as pd
import time
import torch
import torch.nn as nn
import torch.optim as optim

from dataset import Dataset
from datetime import datetime
from model import Autoencoder
from torch.utils.data import DataLoader

In [13]:
def train(model, train_loader, criterion, optimizer, epochs=20):
    
    print('starting model training...')
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"training on {'cuda' if torch.cuda.is_available() else 'cpu'}")
    model.to(device)
    history = []
    
    for epoch in range(epochs):
        start = time.time()
        model.train()
        training_loss, running_loss = 0.0, 0.0
        
        # Iterando sobre o dataset
        for batch_i, data in enumerate(train_loader):
            X = data['X'].to(device)
            
            # Zero Grad
            optimizer.zero_grad()

            # Forward Pass
            output = model(X)

            # Loss Function
            loss = criterion(output, X)

            # Backward Pass
            loss.backward()
            running_loss += loss.item()
            training_loss += loss.item()

            # Update
            optimizer.step()

            if batch_i % 100 == 99:
                print('Batch: {}, Avg. Loss: {}'.format(batch_i + 1, running_loss / 100))
                running_loss = 0.0
        
        # Epoch results
        training_loss /= len(train_loader)
        print(f'[{round(time.time() - start, 3)} secs] Epoch: {epoch+1}/{epochs}', end='')
        print(f', Training loss: {training_loss}', end='\n\n')
        history.append(training_loss)
    
    print('training finished.')
    
    date = str(datetime.now()).split('.')[0]    
    model_path = f'.\\models\\model_{re.sub(r"[^0-9]", "", date)}.pt'
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': training_loss
    }, model_path)
    
    print('model saved.')
    
    return history

def load_model(path, dataset):
    dim = dataset[0]['X'].shape[0]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model = Autoencoder(input_dim=dim)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    train_loss = checkpoint['loss']
    
    return model, optimizer, epoch, train_loss

In [14]:
train_dataset = Dataset(path='.\\data\\training.h5', key='normal')
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=0)

criterion = nn.MSELoss()

## Training new model

In [12]:
dim = train_dataset[0]['X'].shape[0]

model = Autoencoder(input_dim=dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(model)

In [16]:
# Training model
train_loss = train(model, train_loader, criterion, optimizer, epochs=100)

starting model training...
training on cuda
Batch: 100, Avg. Loss: 0.15866318181157113
Batch: 200, Avg. Loss: 0.1587778677046299
Batch: 300, Avg. Loss: 0.1588214559853077
Batch: 400, Avg. Loss: 0.15748187452554702
Batch: 500, Avg. Loss: 0.1570701564848423
Batch: 600, Avg. Loss: 0.15782595291733742
Batch: 700, Avg. Loss: 0.15832166522741317
Batch: 800, Avg. Loss: 0.15788631066679953
Batch: 900, Avg. Loss: 0.15846067562699317
Batch: 1000, Avg. Loss: 0.15777762085199357
Batch: 1100, Avg. Loss: 0.15831221207976343
Batch: 1200, Avg. Loss: 0.15810422986745834
Batch: 1300, Avg. Loss: 0.1590543806552887
Batch: 1400, Avg. Loss: 0.15723520144820213
Batch: 1500, Avg. Loss: 0.15841972038149835
Batch: 1600, Avg. Loss: 0.1580926398932934
Batch: 1700, Avg. Loss: 0.15700656071305275
Batch: 1800, Avg. Loss: 0.15746187850832938
Batch: 1900, Avg. Loss: 0.15811118230223656
[29.509 secs] Epoch: 1/100, Training loss: 0.15803474600416367

Batch: 100, Avg. Loss: 0.15779236942529679
Batch: 200, Avg. Loss: 0.15

Batch: 600, Avg. Loss: 0.15635678485035898
Batch: 700, Avg. Loss: 0.15555530816316604
Batch: 800, Avg. Loss: 0.15750452518463134
Batch: 900, Avg. Loss: 0.15658304944634438
Batch: 1000, Avg. Loss: 0.15621553167700766
Batch: 1100, Avg. Loss: 0.15562134623527527
Batch: 1200, Avg. Loss: 0.15511239528656007
Batch: 1300, Avg. Loss: 0.15585446923971177
Batch: 1400, Avg. Loss: 0.15666435718536376
Batch: 1500, Avg. Loss: 0.15643068492412568
Batch: 1600, Avg. Loss: 0.15753284946084023
Batch: 1700, Avg. Loss: 0.15781215235590934
Batch: 1800, Avg. Loss: 0.15555067256093025
Batch: 1900, Avg. Loss: 0.1569082583487034
[29.356 secs] Epoch: 10/100, Training loss: 0.1564566586300075

Batch: 100, Avg. Loss: 0.15634382054209708
Batch: 200, Avg. Loss: 0.15501427859067918
Batch: 300, Avg. Loss: 0.1566769964993
Batch: 400, Avg. Loss: 0.15608716025948524
Batch: 500, Avg. Loss: 0.15717800661921502
Batch: 600, Avg. Loss: 0.15742026314139365
Batch: 700, Avg. Loss: 0.15692965060472489
Batch: 800, Avg. Loss: 0.156

Batch: 1200, Avg. Loss: 0.15456588089466095
Batch: 1300, Avg. Loss: 0.1543232600390911
Batch: 1400, Avg. Loss: 0.1541843019425869
Batch: 1500, Avg. Loss: 0.1546381551027298
Batch: 1600, Avg. Loss: 0.15382508605718612
Batch: 1700, Avg. Loss: 0.15428530991077424
Batch: 1800, Avg. Loss: 0.15385766983032226
Batch: 1900, Avg. Loss: 0.1550352095067501
[29.411 secs] Epoch: 19/100, Training loss: 0.15425047044286708

Batch: 100, Avg. Loss: 0.15422604322433472
Batch: 200, Avg. Loss: 0.15364638239145278
Batch: 300, Avg. Loss: 0.1546758659183979
Batch: 400, Avg. Loss: 0.1533428616821766
Batch: 500, Avg. Loss: 0.15405820548534394
Batch: 600, Avg. Loss: 0.15466688752174376
Batch: 700, Avg. Loss: 0.1548996214568615
Batch: 800, Avg. Loss: 0.15554433301091194
Batch: 900, Avg. Loss: 0.15481685489416122
Batch: 1000, Avg. Loss: 0.1538917465507984
Batch: 1100, Avg. Loss: 0.1540569543838501
Batch: 1200, Avg. Loss: 0.1526583594083786
Batch: 1300, Avg. Loss: 0.15440940394997596
Batch: 1400, Avg. Loss: 0.1555

Batch: 1700, Avg. Loss: 0.15400921627879144
Batch: 1800, Avg. Loss: 0.15341089516878129
Batch: 1900, Avg. Loss: 0.15388143166899682
[29.4 secs] Epoch: 28/100, Training loss: 0.15352314584788426

Batch: 100, Avg. Loss: 0.15392174944281578
Batch: 200, Avg. Loss: 0.15326053857803346
Batch: 300, Avg. Loss: 0.15325309321284294
Batch: 400, Avg. Loss: 0.153187772333622
Batch: 500, Avg. Loss: 0.15311864241957665
Batch: 600, Avg. Loss: 0.15366484820842743
Batch: 700, Avg. Loss: 0.15337931737303734
Batch: 800, Avg. Loss: 0.15390020594000817
Batch: 900, Avg. Loss: 0.15494948863983155
Batch: 1000, Avg. Loss: 0.15413354098796844
Batch: 1100, Avg. Loss: 0.15388424560427666
Batch: 1200, Avg. Loss: 0.15348630473017694
Batch: 1300, Avg. Loss: 0.15347436740994452
Batch: 1400, Avg. Loss: 0.15434264823794364
Batch: 1500, Avg. Loss: 0.15313375532627105
Batch: 1600, Avg. Loss: 0.1526926089823246
Batch: 1700, Avg. Loss: 0.15358122140169145
Batch: 1800, Avg. Loss: 0.15541292279958724
Batch: 1900, Avg. Loss: 0

Batch: 200, Avg. Loss: 0.15409815564751625
Batch: 300, Avg. Loss: 0.15298907071352005
Batch: 400, Avg. Loss: 0.15383588433265685
Batch: 500, Avg. Loss: 0.15501092240214348
Batch: 600, Avg. Loss: 0.1536938479542732
Batch: 700, Avg. Loss: 0.153838409781456
Batch: 800, Avg. Loss: 0.1538931192457676
Batch: 900, Avg. Loss: 0.15421874582767486
Batch: 1000, Avg. Loss: 0.1543443477153778
Batch: 1100, Avg. Loss: 0.1549333342909813
Batch: 1200, Avg. Loss: 0.1549051833152771
Batch: 1300, Avg. Loss: 0.15315936371684075
Batch: 1400, Avg. Loss: 0.15436507791280746
Batch: 1500, Avg. Loss: 0.15395511507987977
Batch: 1600, Avg. Loss: 0.154713084846735
Batch: 1700, Avg. Loss: 0.1541477282345295
Batch: 1800, Avg. Loss: 0.1548214739561081
Batch: 1900, Avg. Loss: 0.15484688431024551
[29.426 secs] Epoch: 38/100, Training loss: 0.15421183836649238

Batch: 100, Avg. Loss: 0.1543770721554756
Batch: 200, Avg. Loss: 0.15473283872008323
Batch: 300, Avg. Loss: 0.15443502753973007
Batch: 400, Avg. Loss: 0.154193415

...

Batch: 1400, Avg. Loss: 0.1505543890595436
Batch: 1500, Avg. Loss: 0.1518213430047035
Batch: 1600, Avg. Loss: 0.1513649246096611
Batch: 1700, Avg. Loss: 0.1510666672885418
Batch: 1800, Avg. Loss: 0.15080695420503618
Batch: 1900, Avg. Loss: 0.15028389245271684
[29.419 secs] Epoch: 56/100, Training loss: 0.1512160452279122

Batch: 100, Avg. Loss: 0.1502380068600178
Batch: 200, Avg. Loss: 0.15018618673086168
Batch: 300, Avg. Loss: 0.15052672505378722
Batch: 400, Avg. Loss: 0.14999316915869712
Batch: 500, Avg. Loss: 0.15081124886870384
Batch: 600, Avg. Loss: 0.14984161004424096
Batch: 700, Avg. Loss: 0.14958956107497215
Batch: 800, Avg. Loss: 0.14941147178411485
Batch: 900, Avg. Loss: 0.1510418574512005
Batch: 1000, Avg. Loss: 0.15085195660591125
Batch: 1100, Avg. Loss: 0.15204399228096008
Batch: 1200, Avg. Loss: 0.1503410156071186
Batch: 1300, Avg. Loss: 0.15226784631609916
Batch: 1400, Avg. Loss: 0.15086492151021957
Batch: 1500, Avg. Loss: 0.149710513651371
Batch: 1600, Avg. Loss: 0.1513

Batch: 1900, Avg. Loss: 0.1513286243379116
[29.348 secs] Epoch: 65/100, Training loss: 0.15115159383175195

Batch: 100, Avg. Loss: 0.1508090776205063
Batch: 200, Avg. Loss: 0.15167398750782013
Batch: 300, Avg. Loss: 0.151190784573555
Batch: 400, Avg. Loss: 0.1502358441054821
Batch: 500, Avg. Loss: 0.1506217096745968
Batch: 600, Avg. Loss: 0.15095247343182563
Batch: 700, Avg. Loss: 0.15079780220985411
Batch: 800, Avg. Loss: 0.15176899269223212
Batch: 900, Avg. Loss: 0.15173733696341515
Batch: 1000, Avg. Loss: 0.15192886725068092
Batch: 1100, Avg. Loss: 0.15009278669953346
Batch: 1200, Avg. Loss: 0.15112723991274835
Batch: 1300, Avg. Loss: 0.15159200981259346
Batch: 1400, Avg. Loss: 0.15039573952555657
Batch: 1500, Avg. Loss: 0.15127143561840056
Batch: 1600, Avg. Loss: 0.1522854024171829
Batch: 1700, Avg. Loss: 0.15200115263462066
Batch: 1800, Avg. Loss: 0.15203972339630126
Batch: 1900, Avg. Loss: 0.1521287153661251
[29.368 secs] Epoch: 66/100, Training loss: 0.1513241687398728

Batch: 1

Batch: 500, Avg. Loss: 0.15074440866708755
Batch: 600, Avg. Loss: 0.15145018473267555
Batch: 700, Avg. Loss: 0.15113261446356774
Batch: 800, Avg. Loss: 0.15061475202441216
Batch: 900, Avg. Loss: 0.15238463044166564
Batch: 1000, Avg. Loss: 0.15193841233849525
Batch: 1100, Avg. Loss: 0.15088920220732688
Batch: 1200, Avg. Loss: 0.15050218105316163
Batch: 1300, Avg. Loss: 0.15117825150489808
Batch: 1400, Avg. Loss: 0.15075247138738632
Batch: 1500, Avg. Loss: 0.1525949366390705
Batch: 1600, Avg. Loss: 0.15159811183810235
Batch: 1700, Avg. Loss: 0.15116813376545907
Batch: 1800, Avg. Loss: 0.15106527060270308
Batch: 1900, Avg. Loss: 0.15167044818401337
[29.338 secs] Epoch: 75/100, Training loss: 0.15134832573370166

Batch: 100, Avg. Loss: 0.15190708205103876
Batch: 200, Avg. Loss: 0.15214611262083053
Batch: 300, Avg. Loss: 0.15198185786604881
Batch: 400, Avg. Loss: 0.15083048716187478
Batch: 500, Avg. Loss: 0.15047059029340745
Batch: 600, Avg. Loss: 0.14985758408904076
Batch: 700, Avg. Loss: 

Batch: 1200, Avg. Loss: 0.1494721594452858
Batch: 1300, Avg. Loss: 0.15044932052493096
Batch: 1400, Avg. Loss: 0.15062755912542344
Batch: 1500, Avg. Loss: 0.1497802120447159
Batch: 1600, Avg. Loss: 0.14983998998999595
Batch: 1700, Avg. Loss: 0.14987235873937607
Batch: 1800, Avg. Loss: 0.1510931695997715
Batch: 1900, Avg. Loss: 0.1514621366560459
[29.382 secs] Epoch: 84/100, Training loss: 0.15068379459385983

Batch: 100, Avg. Loss: 0.15059604823589326
Batch: 200, Avg. Loss: 0.15118030697107315
Batch: 300, Avg. Loss: 0.15147995233535766
Batch: 400, Avg. Loss: 0.15009935691952705
Batch: 500, Avg. Loss: 0.14947332337498664
Batch: 600, Avg. Loss: 0.14959409922361375
Batch: 700, Avg. Loss: 0.15100446090102196
Batch: 800, Avg. Loss: 0.14989247515797616
Batch: 900, Avg. Loss: 0.14933600008487702
Batch: 1000, Avg. Loss: 0.14942512944340705
Batch: 1100, Avg. Loss: 0.15062185883522033
Batch: 1200, Avg. Loss: 0.1505020585656166
Batch: 1300, Avg. Loss: 0.14919936269521714
Batch: 1400, Avg. Loss: 0

Batch: 1800, Avg. Loss: 0.1495738507807255
Batch: 1900, Avg. Loss: 0.1495245161652565
[29.44 secs] Epoch: 93/100, Training loss: 0.1498841389617144

Batch: 100, Avg. Loss: 0.14986807629466056
Batch: 200, Avg. Loss: 0.15018128156661986
Batch: 300, Avg. Loss: 0.15013656601309777
Batch: 400, Avg. Loss: 0.15053633987903595
Batch: 500, Avg. Loss: 0.150851564258337
Batch: 600, Avg. Loss: 0.15018850311636925
Batch: 700, Avg. Loss: 0.15053235188126565
Batch: 800, Avg. Loss: 0.15064614340662957
Batch: 900, Avg. Loss: 0.15008470222353934
Batch: 1000, Avg. Loss: 0.15049154102802276
Batch: 1100, Avg. Loss: 0.1495251965522766
Batch: 1200, Avg. Loss: 0.14991248592734338
Batch: 1300, Avg. Loss: 0.14962834551930426
Batch: 1400, Avg. Loss: 0.15033922955393791
Batch: 1500, Avg. Loss: 0.1502607937157154
Batch: 1600, Avg. Loss: 0.14963323950767518
Batch: 1700, Avg. Loss: 0.14947276771068574
Batch: 1800, Avg. Loss: 0.15098178133368492
Batch: 1900, Avg. Loss: 0.15014087289571762
[29.387 secs] Epoch: 94/100,

In [17]:
fig = go.Figure(data=go.Scatter(y=train_loss, mode='lines+markers'))
fig.show()

## Loading existing model

In [15]:
# Loading model
model_path = '.\\models\\model_20210604204023.pt'

model, optimizer, epoch, train_loss = load_model(model_path, train_dataset)

In [17]:
train_dataset = Dataset(path='.\\data\\training.h5', key='normal', simulation=1)
mse = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for data in train_dataset:
    X = data['X'].to(device)
    output = model(X)
    mse.append(criterion(output, X).item())
    
fig = go.Figure(data=go.Scatter(y=mse, mode='lines+markers'))
fig.show()

In [27]:
train_faulty_dataset = Dataset(path='.\\data\\training.h5', key='faulty', simulation=(1, 1))

mse_faulty = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for data in train_faulty_dataset:
    X = data['X'].to(device)
    output = model(X)
    mse_faulty.append(criterion(output, X).item())
    
fig = go.Figure(data=go.Scatter(y=mse_faulty, mode='lines+markers'))
fig.show()